# Function‑Calling Demo
Transform user intent into structured arguments and execute a Python function.

In [ ]:
!pip -q install openai ipywidgets

OpenAI's function calling feature lets you register tools that the model can invoke to return structured results.This example shows how to pass a function schema via the `tools` parameter and let ChatGPT automatically call it.

In [ ]:
from openai import OpenAIimport os, json, datetime, ipywidgets as wfrom IPython.display import display, Markdownclient = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))def schedule_meeting(date:str, time:str, topic:str):    dt = datetime.datetime.strptime(f'{date} {time}', '%Y-%m-%d %H:%M')    return f"📅 '{topic}' on {dt.strftime('%A %b %d, %Y at %I:%M %p')}"TOOLS=[{ 'type':'function', 'function':{   'name':'schedule_meeting',   'description':'Schedule a meeting',   'parameters':{     'type':'object',     'properties':{         'date':{'type':'string'},         'time':{'type':'string'},         'topic':{'type':'string'}},     'required':['date','time','topic']}}}]query=w.Text(value='Book a sync about budget next Monday at 14:30.',description='Query:')run_btn=w.Button(description='Run')out=w.Output()def run(_):    with out:        out.clear_output()        res=client.chat.completions.create(model='gpt-4o-mini',            messages=[{'role':'user','content':query.value}],            tools=TOOLS,            temperature=0)        msg=res.choices[0].message        if msg.tool_calls:            args=json.loads(msg.tool_calls[0].function.arguments)            display(Markdown('Args:\n```json\n'+json.dumps(args,indent=2)+'\n```'))            display(Markdown('Result: '+schedule_meeting(**args)))        else:            print('No function call.')run_btn.on_click(run)display(w.VBox([query, run_btn, out]))